Understanding Hired Rides in NYC
Project prompt

This scaffolding notebook may be used to help setup your final project. It's totally optional whether you make use of this or not.

If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish.

Anything in italics (prose) or comments (in code) is meant to provide you with guidance. Remove the italic lines and provided comments before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading.

All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.

Project Setup

In [1]:
#TODO:// 1. bonus: Test needed; [NOT DONE]
# 2. table type and primary key should be checked; [DONE]
# 3. docuement need to be enhanced. [DONE]
# 4. the way download the data?  [DDNE]
# 5.check the correctness of value;  [DONE]
# 6.bonus: sunset table and one more vis? [NOT DONE]

from typing import List, Dict, Any, Tuple
from tqdm import tqdm 
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from ipywidgets import interact
from scipy.stats import sem, t
from ipywidgets import SelectMultiple
from folium.plugins import HeatMap
import bs4
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import requests
import sqlalchemy as db
import ipywidgets as widgets
import seaborn as sns
import geopandas as gpd
import re 
import os
import folium
import warnings
warnings.filterwarnings("ignore")

In [2]:
# any constants you might need; some have been added for you, and 
# some you need to fill in

TLC_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_ZONES_DIR = ""
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
WEATHER_CSV_DIR = "./weather_data"

PARQUET_DIR = "parquet_files"

CRS = 4326  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

engine = create_engine(DATABASE_URL)

In [3]:
# Make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise

Part 1: Data Preprocessing

Load Taxi Zones

In [4]:
def load_taxi_zones(shapefile:str='./taxi_zones.shp') -> gpd.GeoDataFrame :
    """
    Load the taxi zone shapefile into a GeoDataFrame

    Keyword Arguments:
    shapefile {str} -- the path to the shapefile (default: {'./taxi_zones.shp'})

    Returns:
    gpd.GeoDataFrame -- the GeoDataFrame containing the taxi zones
    """
    gdf = gpd.read_file(shapefile).to_crs(4326)
    gdf['latitude'] = gdf.geometry.centroid.y
    gdf['longitude'] = gdf.geometry.centroid.x
    gdf = gdf[['LocationID', 'latitude', 'longitude']]
    return gdf 

In [5]:
gdf = load_taxi_zones()

Calculate Sample Size

In [6]:
def calculate_sample_size(population,z=1.96, e=0.05) -> int:
    """
    calculate the sample size needed for a given population
    :param population: the size of the population
    :param z: the parameter of confidence level
    :param e: the margin of error

    :return: the sample size needed
    """
    n0 = z**2 * 0.5 * 0.5 / e**2
    n = n0 / (1 + (n0 - 1) / population)
    return int(n)

Common Functions

In [7]:
def convert_location_id_to_lat_lon(df:pd.DataFrame) -> pd.DataFrame:
    """
    convert the pickup and dropoff location id to latitude and longitude

    Keyword arguments:
    df -- the dataframe to be converted

    Returns:
    pd.DataFrame -- the dataframe with latitude and longitude columns
    """
    df = df.merge(gdf, left_on='PULocationID', right_on='LocationID', how='left').rename(columns={'latitude':'PULatitude', 'longitude':'PULongitude'}).drop(columns='LocationID')
    df = df.dropna(subset=['PULatitude', 'PULongitude'])
    df = df.merge(gdf, left_on='DOLocationID', right_on='LocationID', how='left').rename(columns={'latitude':'DOLatitude', 'longitude':'DOLongitude'}).drop(columns='LocationID')
    df = df.dropna(subset=['DOLatitude', 'DOLongitude'])
    return df

In [8]:
def filter_taxi_urls(all_urls:List[str]) -> List[str]:
    """
    filter the urls that contain the yellow taxi data

    Keyword arguments:
    all_urls -- the list of urls to be filtered

    Returns:
    List[str] -- the list of filtered urls
    """
    pattern = re.compile(r'(yellow)_tripdata_.*?(202[0-4])-([0-1][0-9])')
    result = []
    for url in all_urls:
        filename = url.split("/")[-1]
        match = pattern.match(filename)
        if match is not None:
            result.append(url)
    return result


def filter_urls(all_urls:List[str],re_pattern :str= '(fhvhv)_tripdata_.*?(202[0-4])-([0-1][0-9])') -> List[str]:
    """
    filter url with given pattern

    Keyword arguments:
    all_urls -- the list of urls to be filtered
    re_pattern -- the pattern to be matched

    Returns:
    List[str] -- the list of filtered urls
    """
    pattern = re.compile(f"{re_pattern}")
    result = []
    for url in all_urls:
        filename = url.split("/")[-1]
        match = pattern.match(filename)
        if match is not None:
            result.append(url)
    return result


Process Taxi Data

In [9]:
def remove_invalid_record_of_taxi_data(df:pd.DataFrame) -> pd.DataFrame:
    """
    remove the data points outside of NYC
    remove the data with a nan pick
    remove the distance less equal than 0
    
    Keyword arguments:

    Returns:
    pd.Dataframe -- the row if it is valid, None otherwise
    """
    valid = (
            (df['PULatitude'].between(NEW_YORK_BOX_COORDS[0][0], NEW_YORK_BOX_COORDS[1][0])) &
            (df['PULongitude'].between(NEW_YORK_BOX_COORDS[0][1], NEW_YORK_BOX_COORDS[1][1])) &
            (df['DOLatitude'].between(NEW_YORK_BOX_COORDS[0][0], NEW_YORK_BOX_COORDS[1][0])) &
            (df['DOLongitude'].between(NEW_YORK_BOX_COORDS[0][1], NEW_YORK_BOX_COORDS[1][1])) &
            (~df['tpep_pickup_datetime'].isna()) &
            (~df['tpep_dropoff_datetime'].isna()) &
            (df['trip_distance'] > 0)
        )
    
    return df[valid]

def get_and_clean_month(url:str)->pd.DataFrame:
    """
    download and clean the data from the given url

    Keyword arguements:
    url -- the url to download the data

    Returns:
    pd.DataFrame -- the cleaned dataframe
    """
    try:
        parquet_file = f"{url.split('/')[-1].strip()}"
        if os.path.exists(f"{PARQUET_DIR}/{parquet_file}"):
            df = pd.read_parquet(f"{PARQUET_DIR}/{parquet_file}")
        else:
            # wget = f"wget {url.strip()} -O {PARQUET_DIR}/{parquet_file}"
            # os.system(wget)

            response = requests.get(url.strip(), stream=True)
            response.raise_for_status()  # Raise an exception for HTTP errors
            file_path = os.path.join(PARQUET_DIR, parquet_file)
            # Save the content to the file
            with open(file_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            df = pd.read_parquet(f"{PARQUET_DIR}/{parquet_file}")
            
        n = calculate_sample_size(df.shape[0])
        df = df.sample(n)
        df = convert_location_id_to_lat_lon(df)
        df = remove_invalid_record_of_taxi_data(df)

        return df
    except Exception as e:
        raise


In [10]:
def get_and_clean_taxi_data(parquet_urls:List[str]) -> pd.DataFrame:
    """
    download and clean the data from the given urls

    Keyword arguements:
    parquet_urls -- the list of urls to download the data

    Returns:
    pd.DataFrame -- the cleaned dataframe
    """

    all_taxi_dataframes = []
    parquet_urls = filter_urls(parquet_urls,'(yellow)_tripdata_.*?(202[0-4])-([0-1][0-9])')
    for parquet_url in tqdm(parquet_urls):
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month(parquet_url)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes)
    return taxi_data

In [11]:
def get_all_urls_from_taxi_page(url:str) -> List[str]:
    """
    get all urls from the page of the given url

    Keyword arguments:
    url -- the url to get all urls from

    Returns:
    List[str] -- the list of all urls
    """
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.content, 'html.parser')
    urls = [a['href'] for a in soup.find_all('a', href=True)]
    return urls


def find_all_parquet_urls(urls:List[str]) -> List[str]:
    """
    filter the all parquet urls 

    Keyword arguments:
    urls -- the list of urls to be filtered

    Returns:
    List[str] -- the list of filtered urls
    """
    parquet_urls = [url for url in urls if 'parquet' in url]
    return parquet_urls

In [12]:
def get_taxi_data() ->pd.DataFrame:
    """ 
    get the taxi data from the TLC website
    
    Returns:
    pd.DataFrame -- the taxi data
    """
    if not os.path.exists(PARQUET_DIR):
        os.mkdir(PARQUET_DIR)
    all_urls = get_all_urls_from_taxi_page(TLC_URL)
    all_parquet_urls = find_all_parquet_urls(all_urls)
    taxi_data = get_and_clean_taxi_data(all_parquet_urls)
    return taxi_data

In [13]:
taxi_data_uncleaned = get_taxi_data()

100%|██████████| 57/57 [00:13<00:00,  4.08it/s]


Normalize Columns

In [14]:
taxi_data_cleaned = taxi_data_uncleaned[['tpep_pickup_datetime','tpep_dropoff_datetime',
                               'trip_distance','fare_amount','extra','mta_tax','improvement_surcharge',
                               'tolls_amount',
                               'PULatitude','PULongitude',
                               'DOLatitude','DOLongitude']]

taxi_data_cleaned['base_fare'] = taxi_data_cleaned['fare_amount'] 
taxi_data_cleaned['tax'] = taxi_data_cleaned['mta_tax'] + taxi_data_cleaned['extra']
taxi_data_cleaned['tolls'] = taxi_data_cleaned['tolls_amount']
taxi_data_cleaned['surcharge'] = taxi_data_cleaned['improvement_surcharge']
taxi_data_cleaned = taxi_data_cleaned.rename(columns={'tpep_pickup_datetime':'trip_pickup_datetime',
                                                      'tpep_dropoff_datetime':'trip_dropoff_datetime',
                                                      'trip_distance':'trip_miles',
                                                      'base_fare':'base_fare',
                                                      'tax':'tax',
                                                      'tolls':'tolls',
                                                      'PULatitude':'pickup_latitude',
                                                      'PULongitude':'pickup_longitude',
                                                      'DOLatitude':'dropoff_latitude',
                                                      'DOLongitude':'dropoff_longitude'})
taxi_data_cleaned = taxi_data_cleaned[['trip_pickup_datetime','trip_dropoff_datetime','trip_miles','base_fare','tax','tolls','surcharge',
                                       'pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude']]
taxi_data = taxi_data_cleaned

In [15]:
taxi_data.head()

,trip_pickup_datetime,trip_dropoff_datetime,trip_miles,base_fare,tax,tolls,surcharge,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
0,2024-01-16 20:39:23,2024-01-16 20:51:41,2.85,14.9,1.5,0.0,1.0,40.758028,-73.977698,40.778766,-73.951010
1,2024-01-12 06:30:27,2024-01-12 06:39:30,2.90,14.2,3.0,0.0,1.0,40.791705,-73.973049,40.753309,-74.004015
2,2024-01-14 10:12:17,2024-01-14 10:29:49,3.77,20.5,0.5,0.0,1.0,40.773633,-73.981532,40.735035,-74.008984
3,2024-01-18 16:46:50,2024-01-18 16:59:43,1.44,12.8,3.0,0.0,1.0,40.736824,-73.984052,40.748497,-73.992438
4,2024-01-07 21:30:16,2024-01-07 21:41:32,1.60,12.1,4.0,0.0,1.0,40.727620,-73.985937,40.735035,-74.008984


In [16]:
taxi_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21210 entries, 0 to 380
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   trip_pickup_datetime   21210 non-null  datetime64[ns]
 1   trip_dropoff_datetime  21210 non-null  datetime64[ns]
 2   trip_miles             21210 non-null  float64       
 3   base_fare              21210 non-null  float64       
 4   tax                    21210 non-null  float64       
 5   tolls                  21210 non-null  float64       
 6   surcharge              21210 non-null  float64       
 7   pickup_latitude        21210 non-null  float64       
 8   pickup_longitude       21210 non-null  float64       
 9   dropoff_latitude       21210 non-null  float64       
 10  dropoff_longitude      21210 non-null  float64       
dtypes: datetime64[ns](2), float64(9)
memory usage: 1.9 MB


In [17]:
taxi_data.describe()

,trip_miles,base_fare,tax,tolls,surcharge,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
count,21210.000000,21210.000000,21210.000000,21210.000000,21210.000000,21210.000000,21210.000000,21210.000000,21210.000000
mean,3.231463,15.154521,1.691122,0.465811,0.551551,40.753815,-73.967144,40.755605,-73.970298
std,4.015866,13.934913,1.541233,4.633344,0.354613,0.031734,0.044452,0.032839,0.036850
min,0.010000,-117.100000,-8.000000,-6.940000,-1.000000,40.571769,-74.167235,40.576961,-74.174000
25%,1.100000,7.200000,0.500000,0.000000,0.300000,40.740439,-73.989845,40.740337,-73.989845
50%,1.800000,10.700000,1.000000,0.000000,0.300000,40.758028,-73.977698,40.758028,-73.977698
75%,3.340000,17.500000,3.000000,0.000000,1.000000,40.773633,-73.961764,40.775932,-73.959635
max,37.900000,157.400000,12.250000,606.990000,1.000000,40.899529,-73.735554,40.899529,-73.735554


### Processing Uber Data

In [18]:
def remove_invalid_records_of_uber_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Remove invalid data points from the Uber dataset.

    Keyword arguments:
    df -- the DataFrame to clean

    Returns:
    pd.DataFrame -- the cleaned DataFrame
    """
    valid = (
        (df['PULatitude'].between(NEW_YORK_BOX_COORDS[0][0], NEW_YORK_BOX_COORDS[1][0])) &
        (df['PULongitude'].between(NEW_YORK_BOX_COORDS[0][1], NEW_YORK_BOX_COORDS[1][1])) &
        (df['DOLatitude'].between(NEW_YORK_BOX_COORDS[0][0], NEW_YORK_BOX_COORDS[1][0])) &
        (df['DOLongitude'].between(NEW_YORK_BOX_COORDS[0][1], NEW_YORK_BOX_COORDS[1][1])) &
        (~df['pickup_datetime'].isna()) &
        (~df['dropoff_datetime'].isna()) &
        (df['trip_miles'] > 0)
    )

    # Return the filtered DataFrame
    return df[valid]


def get_and_clean_uber_month(url:List[str]) -> pd.DataFrame:
    """
    download and clean the data from the given url of uber

    Keyword arguements:
    url -- the url to download the data

    Returns:
    pd.DataFrame -- the cleaned dataframe
    """

    try:
        parquet_file = f"{url.split('/')[-1].strip()}"
        if os.path.exists(f"{PARQUET_DIR}/{parquet_file}"):
            df = pd.read_parquet(f"{PARQUET_DIR}/{parquet_file}")
        else:
            # wget = f"wget {url.strip()} -O {PARQUET_DIR}/{parquet_file}"
            # os.system(wget)

            response = requests.get(url.strip(), stream=True)
            response.raise_for_status()
            file_path = os.path.join(PARQUET_DIR, parquet_file)
            with open(file_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            df = pd.read_parquet(f"{PARQUET_DIR}/{parquet_file}")
            
        n = calculate_sample_size(df.shape[0])
        df = df[df['hvfhs_license_num']=='HV0003']
        df = df.sample(n)
        df = convert_location_id_to_lat_lon(df)
        df = remove_invalid_records_of_uber_data(df)

        return df
    except Exception as e:
        raise


In [19]:
def get_and_clean_uber_data(parquet_urls:List[str]) -> pd.DataFrame:
    """
    download and clean the data from the given urls of uber

    Keyword arguements:
    parquet_urls -- the list of urls to download the data

    Returns:
    pd.DataFrame -- the cleaned dataframe
    """

    all_uber_dataframes = []

    uber_parquet_urls = filter_urls(parquet_urls, re_pattern='(fhvhv)_tripdata_.*?(202[0-4])-([0-1][0-9])')

    for parquet_url in tqdm(uber_parquet_urls):
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_uber_month(parquet_url)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_uber_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    uber_data = pd.concat(all_uber_dataframes)
    return uber_data

In [20]:
def get_uber_data() -> pd.DataFrame :
    """
    get the uber data from the TLC website

    Returns:
    pd.DataFrame -- the uber data
    """

    if not os.path.exists(PARQUET_DIR):
        os.mkdir(PARQUET_DIR)
    all_urls = get_all_urls_from_taxi_page(TLC_URL)
    all_parquet_urls = find_all_parquet_urls(all_urls)
    taxi_data = get_and_clean_uber_data(all_parquet_urls)
    return taxi_data

In [21]:
uber_data_uncleaned = get_uber_data()

100%|██████████| 57/57 [06:40<00:00,  7.02s/it]


### Normalize Columns

In [22]:
uber_data_cleaned = uber_data_uncleaned[['pickup_datetime','dropoff_datetime','trip_miles',
                               'base_passenger_fare','tolls','bcf','sales_tax','congestion_surcharge',
                               'PULatitude','PULongitude',
                               'DOLatitude','DOLongitude']]

uber_data_cleaned['base_fare'] = uber_data_cleaned['base_passenger_fare'] #+ uber_data_cleaned['bcf']
uber_data_cleaned['surcharge'] = uber_data_cleaned['congestion_surcharge']
uber_data_cleaned['tax'] = uber_data_cleaned['sales_tax'] +   uber_data_cleaned['bcf'] # TODO: check if this is correct
uber_data_cleaned['tolls'] = uber_data_cleaned['tolls']
uber_data_cleaned = uber_data_cleaned.rename(columns={
    'trip_miles':'trip_distance',
    'pickup_datetime':'trip_pickup_datetime',
    'dropoff_datetime':'trip_dropoff_datetime',
    'base_fare':'base_fare',
    'tax':'tax',
    'tolls':'tolls',
    'surcharge':'surcharge',
    'PULatitude':'pickup_latitude',
    'PULongitude':'pickup_longitude',
    'DOLatitude':'dropoff_latitude',
    'DOLongitude':'dropoff_longitude'
})

uber_data_cleaned = uber_data_cleaned[['trip_pickup_datetime','trip_dropoff_datetime','trip_distance','base_fare','tax','tolls','surcharge',
                                       'pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude']]

uber_data = uber_data_cleaned

In [23]:
uber_data.head()

,trip_pickup_datetime,trip_dropoff_datetime,trip_distance,base_fare,tax,tolls,surcharge,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
0,2024-01-18 18:47:39,2024-01-18 18:53:08,0.55,7.75,0.90,0.0,0.00,40.827902,-73.869680,40.834165,-73.872289
1,2024-01-22 18:33:37,2024-01-22 19:35:24,12.03,55.75,6.48,0.0,2.75,40.753309,-74.004015,40.647527,-73.882413
2,2024-01-23 12:03:58,2024-01-23 12:09:53,0.72,8.87,1.03,0.0,0.00,40.882157,-73.858948,40.897932,-73.852215
3,2024-01-20 14:57:25,2024-01-20 15:05:22,1.41,11.64,1.35,0.0,0.00,40.624835,-74.029892,40.624835,-74.029892
4,2024-01-03 09:43:10,2024-01-03 09:51:30,0.95,12.46,1.45,0.0,0.00,40.690787,-73.974882,40.695338,-73.986086


In [24]:
uber_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21130 entries, 0 to 385
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   trip_pickup_datetime   21130 non-null  datetime64[ns]
 1   trip_dropoff_datetime  21130 non-null  datetime64[ns]
 2   trip_distance          21130 non-null  float64       
 3   base_fare              21130 non-null  float64       
 4   tax                    21130 non-null  float64       
 5   tolls                  21130 non-null  float64       
 6   surcharge              21130 non-null  float64       
 7   pickup_latitude        21130 non-null  float64       
 8   pickup_longitude       21130 non-null  float64       
 9   dropoff_latitude       21130 non-null  float64       
 10  dropoff_longitude      21130 non-null  float64       
dtypes: datetime64[ns](2), float64(9)
memory usage: 1.9 MB


In [25]:
uber_data.describe()

,trip_distance,base_fare,tax,tolls,surcharge,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude
count,21130.000000,21130.000000,21130.000000,21130.000000,21130.000000,21130.000000,21130.000000,21130.000000,21130.000000
mean,4.407939,21.171729,2.508820,0.635672,1.042168,40.737483,-73.934653,40.737407,-73.934315
std,4.358977,15.748473,1.894407,2.490528,1.329559,0.068836,0.064089,0.069448,0.067343
min,0.120000,-9.870000,0.000000,0.000000,0.000000,40.561994,-74.170887,40.561994,-74.174000
25%,1.550000,10.500000,1.230000,0.000000,0.000000,40.690787,-73.984196,40.690787,-73.984052
50%,2.820000,16.605000,1.940000,0.000000,0.000000,40.737699,-73.948522,40.737699,-73.947442
75%,5.560000,26.477500,3.130000,0.000000,2.750000,40.775932,-73.899735,40.775965,-73.898956
max,37.880000,183.320000,22.460000,46.650000,5.500000,40.899529,-73.726655,40.899529,-73.726655


### Processing Weather Data

In [26]:
def get_all_weather_csvs(directory:str) -> List[str]:
    """
    list all the csv files in the given directory

    Keyword arguments:
    directory -- the directory to list the csv files

    Returns:
    List[str] -- the list of csv files
    """
    return [ f"{directory}/{file}" for file in os.listdir(directory)]

In [27]:
def clean_month_weather_data_hourly(csv_file:str) -> pd.DataFrame:
    """
    clean and collect the weather data hourly

    Keyword arguments:
    csv_file -- the csv file to be cleaned

    Returns:
    pd.DataFrame -- the cleaned dataframe
    """

    try:
        # parse the DATE and fetch the DATE_hour column and select the required columns
        df = pd.read_csv(csv_file)
        df['DATE'] =  pd.to_datetime(df['DATE'])
        df['DATE_hour'] = df['DATE'].dt.strftime('%Y-%m-%d-%H')
        df = df.drop_duplicates(subset='DATE_hour', keep='first')
        df = df[['DATE_hour','HourlyPrecipitation','HourlyWindSpeed']]
        # replace T with 0.01
        df['HourlyPrecipitation'] = df['HourlyPrecipitation'].replace('T', 0.00001)
        return df
    except Exception as e:
        print(csv_file)

In [28]:
def clean_month_weather_data_daily(csv_file:str) -> pd.DataFrame:
    """
    clean and collect the weather data daily

    Keyword arguments:
    csv_file -- the csv file to be cleaned

    Returns:
    pd.DataFrame -- the cleaned dataframe
    """
    try:
        # using the last record of the day to represent the weather of the day
        df = pd.read_csv(csv_file)
        # df = df[df['DailyWeather'].notna()]#[['DailyWeather','DailyAverageDryBulbTemperature']]
        df['DATE'] =  pd.to_datetime(df['DATE'])
        df['DATE_day'] = df['DATE'].dt.strftime('%Y-%m-%d')
        df['DailyPrecipitation'] = df['DailyPrecipitation'].replace('T', 0.00001).astype(float)
        df['DailySnowfall'] = df['DailySnowfall'].replace('T', 0.00001).astype(float)
        df[ 'DailyAverageWindSpeed'] =  df[ 'DailyAverageWindSpeed'].astype(float)
        # Fill missing values in specified columns with their respective column means
        df[['DailyPrecipitation', 'DailySnowfall', 'DailyAverageWindSpeed']] = (
            df[['DailyPrecipitation', 'DailySnowfall', 'DailyAverageWindSpeed']].apply(
                lambda col: col.fillna(col.mean() if col.mean() is not None else 0)
            )
        )

        df = df.drop_duplicates(subset='DATE_day', keep='last')
        df = df[['DATE_day','DailyPrecipitation','DailySnowfall','DailyAverageWindSpeed']]

        return df
    except Exception as e:
        print(csv_file)
        raise


In [29]:
def load_and_clean_weather_data() -> Tuple[pd.DataFrame,pd.DataFrame]:
    """
    load and clean the weather data

    Returns:
    Tuple[pd.DataFrame,pd.DataFrame] -- the cleaned hourly and daily weather data    
    """

    weather_csv_files = get_all_weather_csvs(WEATHER_CSV_DIR)
    
    hourly_dataframes = []
    daily_dataframes = []
        
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

In [30]:
print(list(pd.read_csv('weather_data/2020_weather.csv').columns))

['STATION', 'DATE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'REPORT_TYPE', 'SOURCE', 'HourlyAltimeterSetting', 'HourlyDewPointTemperature', 'HourlyDryBulbTemperature', 'HourlyPrecipitation', 'HourlyPresentWeatherType', 'HourlyPressureChange', 'HourlyPressureTendency', 'HourlyRelativeHumidity', 'HourlySkyConditions', 'HourlySeaLevelPressure', 'HourlyStationPressure', 'HourlyVisibility', 'HourlyWetBulbTemperature', 'HourlyWindDirection', 'HourlyWindGustSpeed', 'HourlyWindSpeed', 'Sunrise', 'Sunset', 'DailyAverageDewPointTemperature', 'DailyAverageDryBulbTemperature', 'DailyAverageRelativeHumidity', 'DailyAverageSeaLevelPressure', 'DailyAverageStationPressure', 'DailyAverageWetBulbTemperature', 'DailyAverageWindSpeed', 'DailyCoolingDegreeDays', 'DailyDepartureFromNormalAverageTemperature', 'DailyHeatingDegreeDays', 'DailyMaximumDryBulbTemperature', 'DailyMinimumDryBulbTemperature', 'DailyPeakWindDirection', 'DailyPeakWindSpeed', 'DailyPrecipitation', 'DailySnowDepth', 'DailySnowfall

In [31]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

In [32]:
hourly_weather_data.head()

,DATE_hour,HourlyPrecipitation,HourlyWindSpeed
0,2020-01-01-00,0.00,8.0
1,2020-01-01-01,0.00,8.0
2,2020-01-01-02,0.00,14.0
3,2020-01-01-03,0.00,11.0
4,2020-01-01-04,0.00,6.0


In [33]:
hourly_weather_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42143 entries, 0 to 11634
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   DATE_hour            42143 non-null  object 
 1   HourlyPrecipitation  38054 non-null  object 
 2   HourlyWindSpeed      38426 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.3+ MB


In [34]:
hourly_weather_data.describe()

,HourlyWindSpeed
count,38426.000000
mean,5.097824
std,16.520609
min,0.000000
25%,3.000000
50%,5.000000
75%,7.000000
max,2237.000000


In [35]:
daily_weather_data.head()

,DATE_day,DailyPrecipitation,DailySnowfall,DailyAverageWindSpeed
24,2020-01-01,0.00000,0.0,8.6
49,2020-01-02,0.00000,0.0,5.4
86,2020-01-03,0.15000,0.0,3.4
144,2020-01-04,0.27000,0.0,4.4
169,2020-01-05,0.00001,0.0,11.3


In [36]:
daily_weather_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1757 entries, 24 to 11638
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DATE_day               1757 non-null   object 
 1   DailyPrecipitation     1757 non-null   float64
 2   DailySnowfall          1757 non-null   float64
 3   DailyAverageWindSpeed  1757 non-null   float64
dtypes: float64(3), object(1)
memory usage: 68.6+ KB


In [37]:
daily_weather_data.describe()

,DailyPrecipitation,DailySnowfall,DailyAverageWindSpeed
count,1757.000000,1757.000000,1757.000000
mean,0.141044,0.038742,5.014147
std,0.408459,0.490489,2.266054
min,0.000000,0.000000,0.600000
25%,0.000000,0.000000,3.300000
50%,0.000000,0.000000,4.800000
75%,0.100000,0.000000,6.200000
max,7.130000,14.800000,14.200000


Part 2: Storing Cleaned Data

In [38]:
engine = db.create_engine(DATABASE_URL)

In [39]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """

DROP TABLE IF EXISTS hourly_weather;
CREATE TABLE hourly_weather  (
    DATE_hour TEXT PRIMARY KEY,
    HourlyPrecipitation REAL,
    HourlyWindSpeed REAL
);

"""

DAILY_WEATHER_SCHEMA = """

DROP TABLE IF EXISTS daily_weather;
CREATE TABLE daily_weather (
    DATE_day TEXT PRIMARY KEY,
    DailyPrecipitation REAL,
    DailySnowfall REAL,
    DailyAverageWindSpeed REAL
);

"""

TAXI_TRIPS_SCHEMA = """


DROP TABLE IF EXISTS taxi_trips;
CREATE TABLE taxi_trips (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    trip_pickup_datetime TEXT,
    trip_dropoff_datetime TEXT,
    trip_miles REAL,
    base_fare REAL,
    tax REAL,
    tolls REAL,
    surcharge REAL,
    pickup_latitude REAL,
    pickup_longitude REAL,
    dropoff_latitude REAL,
    dropoff_longitude REAL
);

"""

UBER_TRIPS_SCHEMA = """

DROP TABLE IF EXISTS uber_trips;
CREATE TABLE uber_trips (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    trip_pickup_datetime TEXT,
    trip_dropoff_datetime TEXT,
    trip_distance REAL,
    base_fare REAL,
    tax REAL,
    tolls REAL,
    surcharge REAL,
    pickup_latitude REAL,
    pickup_longitude REAL,
    dropoff_latitude REAL,
    dropoff_longitude REAL
);

"""

In [40]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [41]:
with open(DATABASE_SCHEMA_FILE, "r") as f:
    sql_script = f.read()

with engine.connect() as connection:
    for statement in sql_script.split(";"):
        statement = statement.strip()
        if statement:  # Skip empty statements
            connection.execute(text(statement))
    

Add Data to Database

In [42]:
def write_dataframes_to_table(table_to_df_dict:Dict[str,pd.DataFrame]):
    """
    write the dataframes to the tables in the database

    Keyword arguments:
    table_to_df_dict -- the dictionary of table name to dataframe mapping
    
    """

    for k,v in table_to_df_dict.items():
        v.to_sql(k, con=engine, if_exists='replace', index=False)

In [43]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data_cleaned,
    "uber_trips": uber_data_cleaned,
    "hourly_weather": hourly_weather_data,
    "daily_weather": daily_weather_data,
}

In [44]:
write_dataframes_to_table(map_table_name_to_dataframe)

Part 3: Understanding the Data

In [45]:
# Helper function to write the queries to file
def write_query_to_file(query:str, outfile:str):
    with open(f"{QUERY_DIRECTORY}/{outfile}", "w") as f:
        f.write(query)

Query 1

In [46]:
QUERY_1_FILENAME = "query1.sql"

QUERY_1 = """
SELECT 
    strftime('%H', trip_pickup_datetime) AS pickup_hour, 
    COUNT(*) AS trip_count
FROM 
    taxi_trips
GROUP BY 
    pickup_hour
ORDER BY 
    trip_count DESC

"""

In [47]:
# execute query either via sqlalchemy
with engine.connect() as con:
    results = con.execute(db.text(QUERY_1)).fetchall()
results

# or via pandas
results1_df = pd.read_sql(QUERY_1, con=engine)
results1_df

,pickup_hour,trip_count
0,18,1498
1,17,1445
2,15,1394
3,14,1366
4,16,1355
5,13,1306
6,12,1300
7,19,1184
8,11,1151
9,10,1089


In [48]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

Query 2

In [49]:
QUERY_2_FILENAME = "query2.sql"

QUERY_2 = """
SELECT 
    strftime('%w', trip_pickup_datetime) AS day_of_week, 
    CASE strftime('%w', trip_pickup_datetime)
        WHEN '0' THEN 'Sunday'
        WHEN '1' THEN 'Monday'
        WHEN '2' THEN 'Tuesday'
        WHEN '3' THEN 'Wednesday'
        WHEN '4' THEN 'Thursday'
        WHEN '5' THEN 'Friday'
        WHEN '6' THEN 'Saturday'
    END AS day_of_week_name,
    COUNT(*) AS trip_count
FROM 
    uber_trips
GROUP BY 
    day_of_week_name
ORDER BY 
    trip_count DESC

"""

In [50]:
# execute query either via sqlalchemy
with engine.connect() as con:
    results = con.execute(db.text(QUERY_2)).fetchall()
results

# or via pandas
results2_df = pd.read_sql(QUERY_2, con=engine)
results2_df

,day_of_week,day_of_week_name,trip_count
0,6,Saturday,3588
1,5,Friday,3384
2,4,Thursday,2962
3,0,Sunday,2922
4,3,Wednesday,2838
5,2,Tuesday,2769
6,1,Monday,2667


In [51]:
write_query_to_file(QUERY_2, QUERY_2_FILENAME)

Query 3

In [52]:
QUERY_3_FILENAME = "query3.sql"

QUERY_3 = """
WITH combined_trips AS (
    SELECT 
        trip_miles AS trip_distance,
        trip_pickup_datetime
    FROM 
        taxi_trips
    WHERE 
        strftime('%Y-%m', trip_pickup_datetime) = '2024-01'
    
    UNION ALL
    
    SELECT 
        trip_distance AS trip_distance,
        trip_pickup_datetime
    FROM 
        uber_trips
    WHERE 
        strftime('%Y-%m', trip_pickup_datetime) = '2024-01'
),
sorted_trips AS (
    SELECT 
        trip_distance,
        ROW_NUMBER() OVER (ORDER BY trip_distance) AS row_num,
        COUNT(*) OVER () AS total_rows
    FROM 
        combined_trips
),
percentile_row AS (
    SELECT 
        trip_distance
    FROM 
        sorted_trips
    WHERE 
        row_num = CAST(0.95 * total_rows AS INTEGER)
)
SELECT 
    trip_distance AS percentile_95
FROM 
    percentile_row;
"""

In [53]:
# execute query either via sqlalchemy
with engine.connect() as con:
    results = con.execute(db.text(QUERY_3)).fetchall()
results

# or via pandas
results3_df = pd.read_sql(QUERY_3, con=engine)
results3_df

,percentile_95
0,13.12


In [54]:
write_query_to_file(QUERY_3, QUERY_3_FILENAME)

### Query 4 

In [55]:
QUERY_4_FILENAME = "query4.sql"

QUERY_4 = """

WITH all_trips AS (
    SELECT 
        strftime('%Y-%m-%d', trip_pickup_datetime) AS trip_date,
        trip_miles AS trip_distance
    FROM 
        taxi_trips
    WHERE 
        strftime('%Y', trip_pickup_datetime) = '2023'

    UNION ALL

    SELECT 
        strftime('%Y-%m-%d', trip_pickup_datetime) AS trip_date,
        trip_distance AS trip_distance
    FROM 
        uber_trips
    WHERE 
        strftime('%Y', trip_pickup_datetime) = '2023'
),
daily_rides AS (
    SELECT 
        trip_date,
        COUNT(*) AS total_rides,
        AVG(trip_distance) AS avg_trip_distance
    FROM 
        all_trips
    GROUP BY 
        trip_date
),
top_days AS (
    SELECT 
        trip_date,
        total_rides,
        avg_trip_distance
    FROM 
        daily_rides
    ORDER BY 
        total_rides DESC
    LIMIT 10
)
SELECT 
    t.trip_date,
    t.total_rides,
    t.avg_trip_distance,
    d.DailyPrecipitation AS avg_daily_precipitation,
    d.DailyAverageWindSpeed AS avg_daily_wind_speed
FROM 
    top_days t
LEFT JOIN 
    daily_weather d
ON 
    t.trip_date = d.DATE_day
ORDER BY 
    t.total_rides DESC;

"""

In [56]:
daily_weather_data[daily_weather_data['DATE_day'] == '2023-07-22']

,DATE_day,DailyPrecipitation,DailySnowfall,DailyAverageWindSpeed
6483,2023-07-22,0.0,0.0,3.1


In [57]:
# execute query either via sqlalchemy
with engine.connect() as con:
    results = con.execute(db.text(QUERY_4)).fetchall()
results

# or via pandas
results4_df=  pd.read_sql(QUERY_4, con=engine)
results4_df

,trip_date,total_rides,avg_trip_distance,avg_daily_precipitation,avg_daily_wind_speed
0,2023-01-21,44,4.190000,0.00001,5.5
1,2023-05-17,44,3.032500,0.00000,5.9
2,2023-11-11,41,5.127561,0.00000,4.0
3,2023-10-13,40,3.902750,0.00000,3.8
4,2023-09-29,39,4.526667,5.48000,8.1
5,2023-02-23,38,3.422632,0.02000,7.3
6,2023-04-29,38,3.022368,2.46000,8.4
7,2023-10-14,38,3.636053,0.81000,4.4
8,2023-02-11,37,4.151892,0.00000,5.9
9,2023-02-25,36,4.369444,0.00001,3.5


In [58]:
write_query_to_file(QUERY_4, QUERY_4_FILENAME)

### Query 5

In [59]:
QUERY_5_FILENAME = "query5.sql"

QUERY_5 = """
WITH snowiest_days AS (
    SELECT 
        DATE_day AS snow_date,
        DailySnowfall
    FROM 
        daily_weather
    WHERE 
        DATE_day BETWEEN '2020-01-01' AND '2024-08-31'
        AND DailySnowfall IS NOT NULL
    ORDER BY 
        DailySnowfall DESC
    LIMIT 10
),
all_trips AS (
    SELECT 
        strftime('%Y-%m-%d', trip_pickup_datetime) AS trip_date
    FROM 
        taxi_trips
    WHERE 
        strftime('%Y-%m-%d', trip_pickup_datetime) BETWEEN '2020-01-01' AND '2024-08-31'

    UNION ALL

    SELECT 
        strftime('%Y-%m-%d', trip_pickup_datetime) AS trip_date
    FROM 
        uber_trips
    WHERE 
        strftime('%Y-%m-%d', trip_pickup_datetime) BETWEEN '2020-01-01' AND '2024-08-31'
),
snow_day_rides AS (
    SELECT 
        s.snow_date,
        s.DailySnowfall,
        COUNT(a.trip_date) AS total_rides
    FROM 
        snowiest_days s
    LEFT JOIN 
        all_trips a
    ON 
        s.snow_date = a.trip_date
    GROUP BY 
        s.snow_date, s.DailySnowfall
    ORDER BY 
        s.DailySnowfall DESC
)
SELECT 
    snow_date,
    DailySnowfall,
    total_rides
FROM 
    snow_day_rides
ORDER BY 
    DailySnowfall DESC;
"""

In [60]:
# execute query either via sqlalchemy
with engine.connect() as con:
    results = con.execute(db.text(QUERY_5)).fetchall()
results

# or via pandas
results5_df = pd.read_sql(QUERY_5, con=engine)
results5_df

,snow_date,DailySnowfall,total_rides
0,2021-02-01,14.8,7
1,2022-01-29,7.3,16
2,2020-12-16,6.5,25
3,2022-01-07,5.8,17
4,2021-02-07,4.5,17
5,2020-12-17,4.0,7
6,2021-02-18,3.2,24
7,2024-02-13,3.2,19
8,2020-01-18,2.1,32
9,2024-02-17,2.0,28


In [61]:
write_query_to_file(QUERY_5, QUERY_5_FILENAME)

### Query 6

In [62]:
QUERY_6_FILENAME = "query6.sql"

QUERY_6 = """
WITH RECURSIVE hours AS (
    -- Generate all hours between 2023-09-25 00:00 and 2023-10-03 23:59
    SELECT datetime('2023-09-25 00:00') AS DATE_hour
    UNION ALL
    SELECT datetime(DATE_hour, '+1 hour')
    FROM hours
    WHERE DATE_hour < '2023-10-03 23:00:00' -- Stop at the last hour
),
hours_format AS (

    SELECT 
        strftime('%Y-%m-%d-%H', DATE_hour) as DATE_hour
    FROM hours as h 
),
rides AS (
    -- Aggregate rides by hour
    SELECT 
        strftime('%Y-%m-%d-%H', trip_pickup_datetime) AS DATE_hour,
        COUNT(*) AS num_rides
    FROM (
        SELECT trip_pickup_datetime FROM taxi_trips
        UNION ALL
        SELECT trip_pickup_datetime FROM uber_trips
    )
    GROUP BY DATE_hour
)
SELECT 
    h.DATE_hour,
    COALESCE(r.num_rides, 0) AS num_rides,
    COALESCE(w.HourlyPrecipitation, 0.0) AS hourly_precipitation,
    COALESCE(w.HourlyWindSpeed, 0.0) AS hourly_wind_speed
FROM 
    hours_format h
LEFT JOIN 
    rides r ON h.DATE_hour = r.DATE_hour
LEFT JOIN 
    hourly_weather w ON h.DATE_hour = w.DATE_hour
ORDER BY 
    h.DATE_hour;
"""

In [63]:
# execute query either via sqlalchemy
with engine.connect() as con:
    results = con.execute(db.text(QUERY_6)).fetchall()
results

# or via pandas
results6_df = pd.read_sql(QUERY_6, con=engine)
results6_df

,DATE_hour,num_rides,hourly_precipitation,hourly_wind_speed
0,2023-09-25-00,0,0.01,8.0
1,2023-09-25-01,0,0.01,6.0
2,2023-09-25-02,0,0.07,7.0
3,2023-09-25-03,1,0.02,0.0
4,2023-09-25-04,0,0.01,6.0
...,...,...,...,...
211,2023-10-03-19,0,0.00,0.0
212,2023-10-03-20,1,0.00,0.0
213,2023-10-03-21,0,0.00,0.0
214,2023-10-03-22,1,0.00,0.0


In [64]:
write_query_to_file(QUERY_6, QUERY_6_FILENAME)